<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/PIPELINE_v0.7b_FB-PROPHET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
==========================================================================================
 PIPELINE v0.7b - PROPHET-MALLIN KOULUTUS JA EVALUOINTI (Sääregressoreilla, Kuvaajien Tallennus)
==========================================================================================

 Versio: 0.7b (Prophet + Regressors + Plot Saving)
 Päivitetty: 2025-04-15 22:05 (EEST)
 Edellinen versio: 0.7 (Prophet Baseline, Korjattu versionhaku)

 Skriptin tarkoitus:
 --------------------
 1. Ladata valmiiksi esikäsitelty data (v0.5).
 2. Valmistella data Prophetille + sääregressorit (Temperature, Visibility).
 3. Jakaa data opetus- ja testijoukkoihin.
 4. Kouluttaa Prophet-malli.
 5. Evaluoida malli ja verrata aiempiin.
 6. Visualisoida ennuste ja komponentit SEKÄ tallentaa kuvaajat images/-kansioon.

 Vaaditut kirjastot: <Samat kuin ennen, varmista prophet ja pyarrow>
"""

# === 1. Alustus ja Kirjastojen Tuonti ===
print("="*60); print(" Vaihe 1: Alustus ja Kirjastojen Tuonti"); print("="*60)
import pandas as pd; import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt; import os; import joblib
try: from prophet import Prophet; import prophet as prophet_module; print(f"--- Prophet-versio: {prophet_module.__version__} ---")
except ImportError: print("VIRHE: Asenna 'prophet'"); Prophet = None
try: import pyarrow; print("'pyarrow' löytyy.")
except ImportError: print("VIRHE: Asenna 'pyarrow'"); pyarrow = None
import requests; import io; import seaborn as sns; plt.style.use('ggplot')
try: import xgboost as xgb; print(f"--- XGBoost-versio: {xgb.__version__} ---")
except ImportError: pass
if Prophet: Prophet.log_level = 'ERROR' # Vähentää Prophetin info-tulosteita

# --- Konfiguraatio ---
INPUT_DATA_URL = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/processed/Helsinki_Data_With_Features_Pipeline_v0.5.parquet"
print(f"Käytetään dataa: {INPUT_DATA_URL}")
TARGET_VARIABLE = 'Ozone'; TEST_DATA_RATIO = 0.2
REGRESSOR_COLUMNS = ['Temperature', 'Visibility'] # Regressorit
IMAGES_FOLDER = "images" # Kansio kuvaajille
OUTPUT_VERSION_TAG = "v0.7b" # Tunniste tallennettaville tiedostoille

# === 2. Datan Lataus ===
# KORJATTU v3: try-except rakenne korjattu
def lataa_esikasitelty_data(url):
    """Lataa esikäsitellyn datan Parquet-tiedostosta URL-osoitteesta."""
    print("\n" + "="*60); print(" Vaihe 2: Datan Lataus"); print("="*60)
    print(f"Yritetään ladata dataa osoitteesta: {url}")
    df = None # Alustetaan df Noneksi
    pyarrow_found = False
    try: import pyarrow; pyarrow_found = True # Tarkistetaan ensin
    except ImportError: print("VIRHE: 'pyarrow' puuttuu."); return None
    if not pyarrow_found: return None # Varmistus

    try: df = pd.read_parquet(url) # Yritetään lataus
    except Exception as e: print(f"VIRHE datan latauksessa: {e}"); return None

    # Jatka vain jos lataus onnistui
    if df is not None:
        print(f"Data ladattu."); print(f"Muoto: {df.shape}")
        if not isinstance(df.index, pd.DatetimeIndex):
            print("VAROITUS: Indeksi ei DatetimeIndex. Muunnetaan...");
            try: df.index = pd.to_datetime(df.index); print("Indeksi muunnettu.")
            except Exception as e_conv: print(f"VIRHE indeksin muunnoksessa: {e_conv}"); return None
        return df
    else: print("VIRHE: DataFrame jäi Noneksi."); return None

# === 3. Datan Valmistelu Prophetille (Regressoreilla) ===
def valmistele_data_prophetille(df, target_sarake, regressor_sarakkeet):
    """Muuntaa DataFramen Prophetin vaatimaan muotoon (ds, y + regressorit)."""
    print("\n" + "="*60); print(" Vaihe 3: Datan Valmistelu Prophetille"); print("="*60)
    if df is None or df.empty: print("VIRHE: Ei dataa."); return None
    required_cols = [target_sarake] + regressor_sarakkeet
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols: print(f"VIRHE: Sarakkeita puuttuu: {missing_cols}"); return None
    if not isinstance(df.index, pd.DatetimeIndex): print("VIRHE: Indeksi ei DatetimeIndex."); return None
    print(f"Valmistellaan data: Target='{target_sarake}', Regressorit={regressor_sarakkeet}")
    df_prophet = df[required_cols].copy().reset_index()
    col_names = df_prophet.columns; original_index_col_name = None
    if 'Timestamp' in col_names: original_index_col_name = 'Timestamp'
    elif 'index' in col_names: original_index_col_name = 'index'
    else:
        potential_ds_cols = col_names.difference(required_cols)
        if len(potential_ds_cols) == 1: original_index_col_name = potential_ds_cols[0]
        else: print(f"VIRHE: Ei tunnistettu aikaleimaa {col_names.tolist()}"); return None
    try:
        df_prophet = df_prophet.rename(columns={original_index_col_name: 'ds', target_sarake: 'y'})
        final_cols = ['ds', 'y'] + regressor_sarakkeet
        if not all(col in df_prophet.columns for col in final_cols): raise ValueError("Uudelleennimeäminen epäonnistui")
        print("Data valmisteltu (sarakkeet 'ds', 'y' + regressorit)."); print(df_prophet.head())
        return df_prophet[final_cols]
    except Exception as rename_e: print(f"VIRHE uudelleennimeämisessä: {rename_e}"); return None

# === 4. Datan Jako (Prophet) ===
def jaa_data_prophet(df_prophet, test_osuus):
    """Jakaa Prophet-muotoisen datan kronologisesti."""
    print("\n" + "="*60); print(" Vaihe 4: Datan Jako (Prophet)"); print("="*60)
    if df_prophet is None or df_prophet.empty: print("VIRHE: Ei dataa."); return None, None
    n=len(df_prophet); split_index=int(n*(1-test_osuus));
    if split_index<=0 or split_index>=n: print(f"VIRHE: Testikoko {test_osuus} virheellinen."); return None,None
    df_train = df_prophet.iloc[:split_index]; df_test = df_prophet.iloc[split_index:]
    print(f"Data jaettu: Opetus {len(df_train)} ({df_train['ds'].min()} - {df_train['ds'].max()})")
    print(f"            Testi  {len(df_test)} ({df_test['ds'].min()} - {df_test['ds'].max()})")
    return df_train, df_test

# === 5. Prophet-mallin Koulutus (Regressoreilla) ===
def kouluta_prophet_malli(df_train, regressor_nimet):
    """Alustaa ja kouluttaa Prophet-mallin, lisäten regressorit."""
    print("\n" + "="*60); print(" Vaihe 5: Prophet-mallin Koulutus (Regressoreilla)"); print("="*60)
    if Prophet is None: print("VIRHE: Prophet puuttuu."); return None
    if df_train is None or df_train.empty: print("VIRHE: Opetusdata puuttuu."); return None
    print("Alustetaan Prophet-malli..."); model = Prophet()
    if regressor_nimet:
        print(f"Lisätään regressorit: {regressor_nimet}")
        for name in regressor_nimet:
            if name in df_train.columns: model.add_regressor(name)
            else: print(f"VAROITUS: Regressoria '{name}' ei löytynyt opetusdatasta."); # Ei lisätä, jos puuttuu
    try: print("Sovitetataan mallia..."); model.fit(df_train); print("Malli koulutettu."); return model
    except Exception as e: print(f"VIRHE koulutuksessa: {e}"); return None

# === 6. Ennustaminen, Evaluointi ja Kuvaajien Tallennus ===
# MUOKATTU: Lisätty kuvaajien tallennus
def ennusta_evaluoi_tallenna_prophet(model, df_test, regressor_nimet, version_tag):
    """Tekee ennusteen, evaluoi ja tallentaa kuvaajat."""
    print("\n" + "="*60); print(f" Vaihe 6: Ennustaminen, Evaluointi & Tallennus ({version_tag})"); print("="*60)
    if model is None or df_test is None or df_test.empty: print("VIRHE: Malli tai testidata puuttuu."); return None
    # Luo future-DataFrame regressoreineen
    future = df_test.drop(columns=['y']); print(f"Luotu 'future' ({future.shape})");
    model_regressors = list(model.extra_regressors.keys()) if hasattr(model, 'extra_regressors') else []
    missing_in_future = [r for r in model_regressors if r not in future.columns]
    if missing_in_future: print(f"VIRHE: Regressorit puuttuvat futuresta: {missing_in_future}"); return None
    # Tee ennuste
    print("Tehdään ennuste..."); forecast = None
    try: forecast = model.predict(future); print("Ennuste laskettu.")
    except Exception as e: print(f"VIRHE ennusteessa: {e}"); return None
    # Laske metriikat
    eval_df = pd.merge(df_test[['ds', 'y']], forecast[['ds', 'yhat']], on='ds')
    y_true = eval_df['y']; y_pred = eval_df['yhat']
    mae = mean_absolute_error(y_true, y_pred); rmse = np.sqrt(mean_squared_error(y_true, y_pred)); r2 = r2_score(y_true, y_pred)
    print(f"\nSuorituskykymetriikat (Testisetti, Regressoreilla):"); print(f"  MAE: {mae:.3f}, RMSE: {rmse:.3f}, R²: {r2:.3f}")
    # Visualisoi ja tallenna
    print("\nVisualisoidaan ja tallennetaan kuvaajat ('images/' kansioon)...");
    images_folder = "images"; os.makedirs(images_folder, exist_ok=True) # Varmista kansion olemassaolo
    try:
        # Ennustekuvaaja
        fig1 = model.plot(forecast); ax1 = fig1.gca(); ax1.plot(eval_df['ds'], eval_df['y'], '.r', alpha=0.5, label='Todelliset (Testi)')
        ax1.set_title(f'Prophet Ennuste vs. Todelliset ({version_tag}, Regressoreilla)'); ax1.set_xlabel('Aika'); ax1.set_ylabel(TARGET_VARIABLE); ax1.legend(loc='upper left')
        forecast_plot_path = os.path.join(images_folder, f"prophet_forecast_{version_tag}.png")
        try: fig1.savefig(forecast_plot_path, bbox_inches='tight'); print(f"  -> Ennustekuvaaja tallennettu: {forecast_plot_path}")
        except Exception as save_e: print(f"  -> VIRHE ennustekuvaajan tallennuksessa: {save_e}")
        plt.show()
        # Komponenttikuvaaja
        print("\nVisualisoidaan mallin komponentit..."); fig2 = model.plot_components(forecast)
        components_plot_path = os.path.join(images_folder, f"prophet_components_{version_tag}.png")
        try: fig2.savefig(components_plot_path, bbox_inches='tight'); print(f"  -> Komponenttikuvaaja tallennettu: {components_plot_path}")
        except Exception as save_e: print(f"  -> VIRHE komponenttikuvaajan tallennuksessa: {save_e}")
        plt.show()
    except Exception as plot_e: print(f"VIRHE visualisoinnissa: {plot_e}")
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

# === Pääsuorituslohko ===
if __name__ == "__main__":
    print("\n" + "="*80); print(f" ALOITETAAN PROPHET PIPELINE {OUTPUT_VERSION_TAG} (REGRESSOREILLA)"); print("="*80)
    df_original, df_prophet_format = None, None; df_train, df_test = None, None
    prophet_model, prophet_results = None, None

    if Prophet is None or not pyarrow:
         print("\nPipeline keskeytetään. Vaadittuja kirjastoja (Prophet tai PyArrow) puuttuu.")
    else:
        try:
            # Vaihe 1: Lataa data (v0.5)
            df_original = lataa_esikasitelty_data(INPUT_DATA_URL)
            if df_original is None: raise ValueError("Datan lataus epäonnistui.")
            # Vaihe 2: Valmistele data Prophetille + Regressorit
            df_prophet_format = valmistele_data_prophetille(df_original, TARGET_VARIABLE, REGRESSOR_COLUMNS)
            if df_prophet_format is None: raise ValueError("Datan valmistelu Prophetille epäonnistui.")
            # Vaihe 3: Jaa data
            df_train, df_test = jaa_data_prophet(df_prophet_format, TEST_DATA_RATIO)
            if df_train is None: raise ValueError("Datan jako epäonnistui.")
            # Vaihe 4: Kouluta malli regressoreilla
            prophet_model = kouluta_prophet_malli(df_train, REGRESSOR_COLUMNS)
            if prophet_model is None: raise ValueError("Prophet-mallin koulutus epäonnistui.")
            # Vaihe 5: Ennusta, evaluoi ja tallenna kuvaajat
            prophet_results = ennusta_evaluoi_tallenna_prophet(prophet_model, df_test, REGRESSOR_COLUMNS, OUTPUT_VERSION_TAG)
            if prophet_results is None: raise ValueError("Prophet-mallin evaluointi epäonnistui.")

            # Vertailu aiempiin malleihin (Lisää v0.5 ja v0.6 tulokset)
            print("\nHaetaan aiempien mallien tulokset vertailua varten...")
            # !! MUISTA PÄIVITTÄÄ NÄMÄ !!
            baseline_mae = 12.804; baseline_rmse = 16.872; baseline_r2 = 0.331
            xgboost_mae = 8.807; xgboost_rmse = 10.764; xgboost_r2 = 0.565
            if baseline_mae is not None and xgboost_mae is not None and prophet_results is not None:
                print("\n" + "-"*60); print(" Vertailu aiempiin malleihin"); print("-"*60)
                print(f"Metriikka        | Baseline (LR) | XGBoost (v0.6) | Prophet ({OUTPUT_VERSION_TAG} R)")
                print(f"-----------------|---------------|----------------|-----------------")
                print(f"MAE              | {baseline_mae:>13.3f} | {xgboost_mae:>14.3f} | {prophet_results['MAE']:>15.3f}")
                print(f"RMSE             | {baseline_rmse:>13.3f} | {xgboost_rmse:>14.3f} | {prophet_results['RMSE']:>15.3f}")
                print(f"R²               | {baseline_r2:>13.3f} | {xgboost_r2:>14.3f} | {prophet_results['R2']:>15.3f}")
                print("-"*60)
            else: print("\nAiempien mallien tai Prophetin tuloksia puuttuu, vertailua ei tehdä täysin.")

        except Exception as e:
            print("\n" + "!"*80); print(f" KRIITTINEN VIRHE PIPELINEN SUORITUKSESSA: {e}");
            import traceback; traceback.print_exc(); print("!"*80)
        finally:
            print("\n" + "="*80); print(f" PROPHET PIPELINE {OUTPUT_VERSION_TAG} SUORITETTU LOPPUUN"); print("="*80)